<a href="https://colab.research.google.com/github/talefty123/testing_code/blob/main/Python/Structured/H_structured_ISD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proposed Model

# **Biblioheque**

In [1]:
import random
import numpy as np
from scipy.stats import rice
# import pandas as pd
import pickle
import torch
import torch.nn as nn
from torch.nn import functional as F
import sys
import timeit
import os

torch.set_default_tensor_type(torch.cuda.DoubleTensor)
torch.set_default_dtype(torch.float64)

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


# class to save results in file

In [2]:
class Record:
    def __init__(self, TextName):
        self.out_file = open(TextName, 'a')
        self.old_stdout = sys.stdout
        sys.stdout = self

    def write(self, text):
        self.old_stdout.write(text)
        self.out_file.write(text)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self.old_stdout

# **slicer the data**

In [3]:
def slicer(data):
    dataI = data[slice(0, len(data), 2)]
    dataQ = data[slice(1, len(data), 2)]
    return(dataI, dataQ)

# **Modulation**

In [4]:
def mapper_16QAM(QAM16, data):
    map0 = 2*data[slice(0, len(data), 2)] + data[slice(1, len(data), 2)]
    map0 = list(map(int, map0))
    dataMapped = []
    for i in range(len(map0)):
        dataMapped.append(QAM16[map0[i]])
    return(dataMapped)

In [5]:
def calculate_bits(Modulation,NumSubcarriers,NumDataSymb):
    if Modulation=='QPSK':
        Nbpscs=2
    elif Modulation=='16QAM':
        Nbpscs=4
    return NumDataSymb*NumSubcarriers*Nbpscs


# **generate noise**

In [6]:
def AWGN(IFsig, SNR):
    dP = np.zeros(len(IFsig))
    P = 0

    for i in range(len(IFsig)):
        dP[i] = abs(IFsig[i])**2
        P = P + dP[i]

    P = P/len(IFsig)
    gamma = 10**(SNR/10)
    N0 = P/gamma
    n = ((N0/2)**(0.5))*np.random.standard_normal(len(IFsig))
    IF_n = np.zeros((len(IFsig),1))

    for i in range(len(IFsig)):
        IF_n[i,:] = IFsig[i] + n[i]

    return(IF_n)

# Generate channel model

In [7]:
def unstructured_channel(Nt, Nr, type):
    if (type == 'gauss'):
        return (np.random.normal(size=(Nt, Nr))+1j*np.random.normal(size=(Nt, Nr)))/np.sqrt(2)

def Generate_channel(Nt, Nr, d_H, array_type, elements_nor, type, Nr_ULA, Nr_UCA):
    if (type == 'gauss'):

        H = (np.random.normal(size=(Nt, Nr))+1j*np.random.normal(size=(Nt, Nr)))/np.sqrt(2)
        H_oG = np.zeros((Nt, Nr), dtype=complex)
        Gamma = []
        Zoa = []
        Aoa = []

        Steering = np.zeros((Nt, Nr, d_H), dtype=complex)

        for nt in range (Nt):
            # gamma = (np.random.normal(size=d_H)+1j*np.random.normal(size=d_H))/np.sqrt(2)
            zoa = np.random.uniform(-np.pi / 2, np.pi / 2, d_H)
            aoa = np.random.uniform(-np.pi / 2, np.pi / 2, d_H)

            # Gamma.append(gamma)
            Zoa.append(zoa)
            Aoa.append(aoa)

        # Gamma = np.array(Gamma)
        Zoa = np.array(Zoa)
        Aoa = np.array(Aoa)

        if array_type == 'ULA':
            for nt in range (Nt):
                for nr in range (Nr):
                    h = 0
                    for dh in range(d_H):
                        r_x = np.sin(Zoa[nt,dh]) * np.cos(Aoa[nt,dh])
                        r_y = np.sin(Zoa[nt,dh]) * np.sin(Aoa[nt,dh])
                        r_z = np.cos(Zoa[nt,dh])

                        # h = h + Gamma[nn,dh] * np.exp(-1j*2*np.pi * (elements_nor[0, 0, nr]*r_x
                        #                                             + elements_nor[1, 0, nr]*r_y
                        #                                             + elements_nor[2, 0, nr]*r_z))

                        Steering[nt, nr, dh] = np.exp(-1j*2*np.pi * (elements_nor[0, 0, nr]*r_x
                                                                    + elements_nor[1, 0, nr]*r_y
                                                                    + elements_nor[2, 0, nr]*r_z))

                    H_oG[nt, nr] = np.divide(H[nt, nr],  Steering[nt, nr, 0])

                    # H[nr, nn] = h
        else:
            for nt in range (Nt):
                r = -1
                for Nr_ULA_index in range (Nr_ULA):
                    for Nr_UCA_index in range (Nr_UCA):
                        r=r+1
                        h = 0
                        for dh in range(d_H):
                            r_x = np.sin(Zoa[nt, dh]) * np.cos(Aoa[nt, dh])
                            r_y = np.sin(Zoa[nt, dh]) * np.sin(Aoa[nt, dh])
                            r_z = np.cos(Zoa[nt, dh])

                            # h = h + Gamma[nt, dh] * np.exp(-1j*2*np.pi *  (elements_nor[0, Nr_ULA_index, Nr_UCA_index]*r_x
                            #                                             + elements_nor[1, Nr_ULA_index, Nr_UCA_index]*r_y
                            #                                             + elements_nor[2, Nr_ULA_index, Nr_UCA_index]*r_z))

                            Steering[nt, r, dh] = np.exp(-1j*2*np.pi *(elements_nor[0, Nr_ULA_index, Nr_UCA_index]*r_x
                                                                        + elements_nor[1, Nr_ULA_index, Nr_UCA_index]*r_y
                                                                        + elements_nor[2, Nr_ULA_index, Nr_UCA_index]*r_z))

                        # tmp_1 = 0
                        # gamma = 0
                        # while(tmp_1 < 0.6):
                        #     gamma = (np.random.normal(size=d_H - 1)+1j*np.random.normal(size=d_H - 1))/np.sqrt(2)
                        #     tmp = 0
                        #     for jj in range (d_H - 1):
                        #         tmp += gamma[jj] * Steering[r, nt, jj]
                        #     last_gamma = np.divide(H[r, nt] - tmp, Steering[r, nt, d_H - 1])
                        #     gamma = np.append(gamma, last_gamma)

                        #     tmp_1 = min(abs(gamma))

                        H_oG[nt, r] = np.divide(H[nt, r],  Steering[nt, r, 0])
                        # H[r, nt] = h

        return H, H_oG, Gamma, Steering

    if (type == 'rayleigh'):
        return (np.random.rayleigh(scale=(1/np.sqrt(2)), size=(Nt, Nr)) + 1j*np.random.rayleigh(scale=(1/np.sqrt(2)), size=(Nt, Nr)))/np.sqrt(2)
    if (type == 'rician'):
        b = 1/np.sqrt(2)
        return (rice.rvs(b, size=(Nt, Nr)) + 1j*rice.rvs(b, size=(Nt, Nr)))/np.sqrt(2)

# **Generate Dataset**



In [8]:
DataSet_x   = []  # x dataset after modulation
DataSet_y   = []  # y dataset
DataSet_HH  = []  # H dataset
DataSet_b   = []  # binary dataset
SNR_min_dB  = 0
SNR_max_dB  = 20
step_dB     = 5
num_dB      = int((SNR_max_dB - SNR_min_dB) / step_dB) + 1

SNR         = np.linspace(SNR_min_dB, SNR_max_dB, num=num_dB)


Nt = 8             # Tx: 8
Nr = 64            # Rx: 128
N_samp = 4000


def Gen_dataset(mode, array_type, snr, imperfect, N_samp):
    DataSet_x   = []  # x dataset after modulation
    DataSet_y   = []  # y dataset
    DataSet_H   = []
    DataSet_HH  = []
    DataSet_Steering = []

    NumSubcarriers = 1
    Modulation = '16QAM'
    QAM16 = [-1, -0.333, 0.333, 1]
    NumDataSymb = 1
    N_type = 'gauss'

    d_H = 1
    d_ULA_nor  = 0.5
    d_UCA_nor  = 0.5
    Nr_UCA = 16
    Nr_ULA = 4

    if array_type == 'ULA':
        elements_nor = np.zeros((3, 1, Nr), dtype=float)
        for Nr_index in range (Nr):
            elements_nor[0, 0, Nr_index] = (Nr_index-1) * d_UCA_nor
            elements_nor[1, 0, Nr_index] = 0
            elements_nor[2, 0, Nr_index] = 0
    else:
        elements_nor = np.zeros((3, Nr_ULA, Nr_UCA), dtype=float)

        R_nor = 0.5 * d_UCA_nor / np.sin(np.pi/Nr_UCA)

        for Nr_ULA_index in range (Nr_ULA):
            for Nr_UCA_index in range (Nr_UCA):
                elements_nor[0, Nr_ULA_index, Nr_UCA_index] = R_nor * np.sin((Nr_UCA_index-1)*(2*np.pi/Nr_UCA)) ;
                elements_nor[1, Nr_ULA_index, Nr_UCA_index] = R_nor * np.cos((Nr_UCA_index-1)*(2*np.pi/Nr_UCA)) ;
                elements_nor[2, Nr_ULA_index, Nr_UCA_index] = (Nr_ULA_index-1) * d_ULA_nor;

    if mode == 'train':
        for snr in SNR:
            for runIdx in range(0, N_samp):      # ! 20000 x Nt: samples
                H, H_oG, Gamma, Steering = Generate_channel(Nt, Nr, d_H, array_type, elements_nor, N_type, Nr_ULA, Nr_UCA)

                HH = np.concatenate((np.concatenate((H_oG.real, H_oG.imag), axis=1),
                                    np.concatenate((-H_oG.imag, H_oG.real), axis=1)), axis=0)

                x = np.zeros((2*Nt, NumSubcarriers))
                a = calculate_bits(Modulation, NumSubcarriers, NumDataSymb)
                DataRaw = np.zeros((Nt, a))
                for t in range(Nt):
                    #"data symbol generate"
                    NumBits = calculate_bits(Modulation, NumSubcarriers, NumDataSymb)
                    bit = np.random.randint(1, 3, NumBits)-1
                    DataRaw[t, :] = bit
                    for j in range(4):
                        DataSet_b.append(bit[j])
                    I = np.zeros((1, a))
                    I[0, :] = DataRaw[t, :]
                    (dataI, dataQ) = slicer(I[0])

                    # Mapper
                    mapI = mapper_16QAM(QAM16, dataI)
                    mapQ = mapper_16QAM(QAM16, dataQ)
                    x[t] = mapI[0]
                    x[t+Nt] = mapQ[0]

                # transpose
                x = x.transpose()

                y_wo_noise = np.matmul(x, HH)

                # noise
                noise = AWGN(y_wo_noise.transpose(), snr)

                y = y_wo_noise + noise.transpose()

                DataSet_x.append(x)    # ! I, Q sample distance by Nt.
                DataSet_y.append(y)                 # ! output sample

                # Imperfect channel: 5%
                # coef = (2*np.random.randint(0,2,size=HH.shape) - 1)
                # HH = HH + coef * HH * 0.05
                DataSet_Steering.append(np.squeeze(Steering))
                DataSet_HH.append(HH)
                DataSet_H.append(H)               # ! Generated channel

    else:
        for runIdx in range(0, N_samp):      # ! 20000 x Nt: samples
            H, H_oG, Gamma, Steering = Generate_channel(Nt, Nr, d_H, array_type, elements_nor, N_type, Nr_ULA, Nr_UCA)

            HH = np.concatenate((np.concatenate((H_oG.real, H_oG.imag), axis=1),
                                np.concatenate((-H_oG.imag, H_oG.real), axis=1)), axis=0)

            x = np.zeros((2*Nt, NumSubcarriers))
            a = calculate_bits(Modulation, NumSubcarriers, NumDataSymb)
            DataRaw = np.zeros((Nt, a))
            for t in range(Nt):
                #"data symbol generate"
                NumBits = calculate_bits(Modulation, NumSubcarriers, NumDataSymb)
                bit = np.random.randint(1, 3, NumBits)-1
                DataRaw[t, :] = bit
                for j in range(4):
                    DataSet_b.append(bit[j])
                I = np.zeros((1, a))
                I[0, :] = DataRaw[t, :]
                (dataI, dataQ) = slicer(I[0])

                # Mapper
                mapI = mapper_16QAM(QAM16, dataI)
                mapQ = mapper_16QAM(QAM16, dataQ)
                x[t] = mapI[0]
                x[t+Nt] = mapQ[0]

            # transpose
            x = x.transpose()

            y_wo_noise = np.matmul(x, HH)

            # noise
            noise = AWGN(y_wo_noise.transpose(), snr)

            y = y_wo_noise + noise.transpose()

            DataSet_x.append(x)    # ! I, Q sample distance by Nt.
            DataSet_y.append(y)                 # ! output sample

            # Imperfect channel: 5%
            DataSet_Steering.append(np.squeeze(Steering))
            DataSet_HH.append(HH)
            DataSet_H.append(H)               # ! Generated channel


    # Shuffle dataset
    random.seed(1)
    temp = list(zip(DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering))
    random.shuffle(temp)
    DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering = zip(*temp)

    return DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering

In [9]:
def reconstruct_channel (H):
# H_raw = [R(H) I(H); -I(H) R(H)]
# we have four version of H_est
    H_est_1 = []
    H_est_2 = []
    H_est_3 = []
    H_est_4 = []

    H_est_Re_1 = H[0:Nt, 0:Nr]
    H_est_Im_1 = H[0:Nt, Nr:2*Nr]
    H_est_Im_2 = - H[Nt:2*Nt, 0:Nr]
    H_est_Re_2 = H[Nt:2*Nt, Nr:2*Nr]

    H_est_1 = H_est_Re_1 + 1j * H_est_Im_1
    H_est_2 = H_est_Re_1 + 1j * H_est_Im_2
    H_est_3 = H_est_Re_2 + 1j * H_est_Im_1
    H_est_4 = H_est_Re_2 + 1j * H_est_Im_2

    return H_est_1, H_est_2, H_est_3, H_est_4

In [10]:
# def NMSE(H_est, H_raw):
#     H_est_1, H_est_2, H_est_3, H_est_4 = reconstruct_channel(H_est)
#     H_est_vec_1 = torch.reshape(H_est_1, [Nt * Nr, 1])
#     H_est_vec_2 = torch.reshape(H_est_2, [Nt * Nr, 1])
#     H_est_vec_3 = torch.reshape(H_est_3, [Nt * Nr, 1])
#     H_est_vec_4 = torch.reshape(H_est_4, [Nt * Nr, 1])

#     H_raw_vec = torch.reshape(H_raw, [Nt * Nr, 1])

#     mse_1       = (torch.norm(H_raw_vec - H_est_vec_1)**2) / len(H_raw_vec)
#     mse_2       = (torch.norm(H_raw_vec - H_est_vec_2)**2) / len(H_raw_vec)
#     mse_3       = (torch.norm(H_raw_vec - H_est_vec_3)**2) / len(H_raw_vec)
#     mse_4       = (torch.norm(H_raw_vec - H_est_vec_4)**2) / len(H_raw_vec)

#     sigEner   = torch.norm(H_raw_vec)**2

#     nmse_1      = mse_1 / sigEner
#     nmse_2      = mse_2 / sigEner
#     nmse_3      = mse_3 / sigEner
#     nmse_4      = mse_4 / sigEner

#     # Best nmse
#     nmse        = min([nmse_1, nmse_2, nmse_3, nmse_4])

#     return torch.abs(nmse)

In [11]:
def NMSE(H_est, H_raw):
    H_est_1, H_est_2, H_est_3, H_est_4 = reconstruct_channel(H_est)

    # Lấy phần thực của các tensor nếu chúng là complex
    H_est_vec_1 = torch.reshape(H_est_1, [Nt * Nr, 1]).abs()
    H_est_vec_2 = torch.reshape(H_est_2, [Nt * Nr, 1]).abs()
    H_est_vec_3 = torch.reshape(H_est_3, [Nt * Nr, 1]).abs()
    H_est_vec_4 = torch.reshape(H_est_4, [Nt * Nr, 1]).abs()

    H_raw_vec = torch.reshape(H_raw, [Nt * Nr, 1]).abs()

    mse_1 = (torch.norm(H_raw_vec - H_est_vec_1)**2) / len(H_raw_vec)
    mse_2 = (torch.norm(H_raw_vec - H_est_vec_2)**2) / len(H_raw_vec)
    mse_3 = (torch.norm(H_raw_vec - H_est_vec_3)**2) / len(H_raw_vec)
    mse_4 = (torch.norm(H_raw_vec - H_est_vec_4)**2) / len(H_raw_vec)

    sigEner = torch.norm(H_raw_vec)**2

    nmse_1 = mse_1 / sigEner
    nmse_2 = mse_2 / sigEner
    nmse_3 = mse_3 / sigEner
    nmse_4 = mse_4 / sigEner

    # Chọn NMSE tốt nhất
    nmse = min([nmse_1, nmse_2, nmse_3, nmse_4])

    return torch.abs(nmse)

In [12]:
def Input_ISDNN(mode, DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering, N_samp):
    H_in = []        # ! H_in    , np.diag(np.diag()) return a diag matrix instead of diag components.
    H_true = []   # ! generated s
    H_raw = []
    # y = []
    e = []        # ! vector errors
    xTx = []
    xTy = []
    Di = []
    steering = [] # ! Steering vector: ZoA and AoA

    if mode == 'train':
        n_sample = N_samp * len(SNR)
    else:
        n_sample = N_samp

    for i in range (n_sample):
        H_true.append(torch.tensor(DataSet_HH[i]))
        H_raw.append(torch.tensor(DataSet_H[i]))
        # y.append(torch.tensor(DataSet_y[i]))
        Di.append(torch.tensor(np.linalg.pinv(np.diag(np.diag(np.dot(DataSet_x[i].transpose(), DataSet_x[i]))))))
        xTy.append(torch.tensor(np.dot(DataSet_x[i].transpose(), DataSet_y[i])))
        H_in.append(torch.matmul(Di[i], xTy[i]))
        e.append(torch.rand([2*Nt, 2*Nr]))
        xTx.append(torch.tensor(np.dot(DataSet_x[i].transpose(), DataSet_x[i])))
        steering.append(torch.tensor(DataSet_Steering[i]))

    H_true = torch.stack(H_true, dim=0)
    H_raw = torch.stack(H_raw, dim=0)
    H_in = torch.stack(H_in, dim=0)
    # y = torch.stack(y, dim=0)
    e = torch.stack(e, dim=0)
    xTx = torch.stack(xTx, dim=0)
    xTy = torch.stack(xTy, dim=0)
    Di = torch.stack(Di, dim=0)
    steering = torch.stack(steering, dim=0)

    return H_true, H_raw, H_in, e, xTx, xTy, Di, steering

# Model

In [13]:
class xv(nn.Module):
    def __init__(self):
        super(xv, self).__init__()

        self.alpha1 = torch.nn.parameter.Parameter(torch.rand(1))
        self.alpha2 = torch.nn.parameter.Parameter(torch.tensor([0.5]))

    def forward(self, Di, H, e, xTx, xTy):

        xTxH = torch.bmm(xTx, H)

        z    = H + torch.bmm(Di, torch.sub(xTy, xTxH)) + self.alpha1 * e

        e    = torch.sub(xTy, xTxH)

        H    = torch.add((1 - self.alpha2) * z, self.alpha2 * H)

        return H, e

In [14]:
class model_driven(nn.Module):
    def __init__(self):
        super(model_driven, self).__init__()
        self.fc1 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc2 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc3 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc4 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc5 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc6 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc7 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc8 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc9 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc10 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc11 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc12 = torch.nn.Linear(2*Nr, 2*Nr)



        self.layer1=xv()
        self.layer2=xv()
        self.layer3=xv()
        self.layer4=xv()
        self.layer5=xv()
        self.layer6=xv()


    def forward(self, Di, H_in, e, xTx, xTy):
        e = self.fc1(e)
        e = self.fc2(e)
        H, e = self.layer1(Di, H_in, e, xTx, xTy)
        H = torch.tanh(H)

        e = self.fc3(e)
        e = self.fc4(e)
        H, e = self.layer2(Di, H, e, xTx, xTy)
        H = torch.tanh(H)

        e = self.fc5(e)
        e = self.fc6(e)
        H, e = self.layer3(Di, H, e, xTx, xTy)
        H = torch.tanh(H)

        e = self.fc7(e)
        e = self.fc8(e)
        H, e = self.layer4(Di, H, e, xTx, xTy)
        H = torch.tanh(H)

        e = self.fc9(e)
        e = self.fc10(e)
        H, e = self.layer5(Di, H, e, xTx, xTy)
        H = torch.tanh(H)

        e = self.fc11(e)
        e = self.fc12(e)
        H, e = self.layer6(Di, H, e, xTx, xTy)

        return H, e

# Define model, optimizer, and loss function

In [15]:
def def_model():
    model = model_driven()
    loss = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    folder_model = './model/'

    if not os.path.isdir(folder_model):
        os.makedirs(folder_model)

    file_model = folder_model + 'H'
    # if os.path.isfile(file_model):
    #     generator = torch.load(file_model)

    record_file = 'H'
    return model, loss, optimizer, record_file, file_model

# Main program

In [16]:
epoch         = 0
expected_epoch = 20000
num_samp      = N_samp * len(SNR)
best_nmse     = 1e9
early_stop    = 0
best_model    = ''
batch_size    = int(num_samp / 512)
# DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering = Gen_dataset('train', 'UCyA', 0, 0, N_samp)
# H_true, H_raw, H_in, e, xTx, xTy, Di, steering = Input_ISDNN('train', DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering, N_samp)

# Kiểm tra nếu file tĩnh tồn tại
if os.path.exists('dataset_SISDNN.pkl'):
    with open('dataset_SISDNN.pkl', 'rb') as f:
        DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering, H_true, H_raw, H_in, e, xTx, xTy, Di, steering = pickle.load(f)
    print("Dữ liệu đã được tải từ file tĩnh!")
else:
    # Sinh dữ liệu nếu file tĩnh không tồn tại
    DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering = Gen_dataset('train', 'UCyA', 0, 0, N_samp)
    H_true, H_raw, H_in, e, xTx, xTy, Di, steering = Input_ISDNN('train', DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering, N_samp)

    # Lưu dữ liệu để lần sau không phải sinh lại
    with open('dataset_SISDNN.pkl', 'wb') as f:
        pickle.dump((DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering, H_true, H_raw, H_in, e, xTx, xTy, Di, steering), f)
    print("Dữ liệu đã được sinh và lưu lại!")

print("Begin training...")
starttime = timeit.default_timer()

while(True):
        epoch = epoch + 1

        init_loss = 1e9
        while( epoch == 1 and init_loss > 150):

                model, loss, optimizer, record_file, file_model = def_model()
                for bs in range (int(num_samp / batch_size)):
                    H_1, e_1 = model(Di[0 + batch_size * bs:batch_size * (bs+1), :, :],
                                 H_in[0 + batch_size * bs:batch_size * (bs+1), :, :],
                                 e[0 + batch_size * bs:batch_size * (bs+1), :, :],
                                 xTx[0 + batch_size * bs:batch_size * (bs+1), :, :],
                                 xTy[0 + batch_size * bs:batch_size * (bs+1), :, :])   # predict output from the model
                    init_loss = loss(H_1, H_true[0 + batch_size * bs:batch_size * (bs+1), :, :]).item()
                    print(init_loss)

        optimizer.zero_grad()   # zero the parameter gradients
        train_loss = 0
        H_f = torch.empty([num_samp, 2*Nt, 2*Nr])
        for bs in range (int(num_samp / batch_size)):
                H_o, e_o = model(Di[0 + batch_size * bs:batch_size * (bs+1), :, :],
                                 H_in[0 + batch_size * bs:batch_size * (bs+1), :, :],
                                 e[0 + batch_size * bs:batch_size * (bs+1), :, :],
                                 xTx[0 + batch_size * bs:batch_size * (bs+1), :, :],
                                 xTy[0 + batch_size * bs:batch_size * (bs+1), :, :])   # predict output from the model
                H_f[0 + batch_size * bs:batch_size * (bs+1), :, :] = H_o
                train_loss = loss(H_o,
                                  H_true[0 + batch_size * bs:batch_size * (bs+1), :, :])   # calculate loss for the predicted output
                train_loss.backward()   # backpropagate the loss
                optimizer.step()        # adjust parameters based on the calculated gradients

        if (epoch % 100 == 0 or epoch == 1):
                nmse = 0
                for j in range (num_samp):
                        nmse += NMSE(H_f[j], H_raw[j])
                nmse = nmse / num_samp

                if (nmse <= best_nmse):
                        torch.save(model.state_dict(), file_model + '_' + str(epoch) + '.pth')
                        best_model = file_model + '_' + str(epoch) + '.pth'
                        best_nmse = nmse
                        early_stop = 0
                else:
                        early_stop += 1

                if (nmse > best_nmse and early_stop == 3):
                        with Record(record_file + '_log.txt'):
                                print(epoch, nmse.item(), train_loss.item())
                                print(str(timeit.default_timer()-starttime))
                        break

                with Record(record_file + '_log.txt'):
                        print(epoch, nmse.item(), train_loss.item())

        if epoch  == expected_epoch:
                torch.save(model.state_dict(), file_model + '_' + str(epoch) + '.pth')
                best_model = file_model + '_' + str(epoch) + '.pth'
                with Record(record_file + '_log.txt'):
                        print("epoch:\n", epoch)
                        print("Latest NMSE:\n", nmse.item())
                        print("Latest Loss:\n", train_loss.item())
                        print(str(timeit.default_timer()-starttime))

                break

<ipython-input-6-3e5f3c45b18a>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dP[i] = abs(IFsig[i])**2


Dữ liệu đã được sinh và lưu lại!
Begin training...
143.88541794228843
144.51302759830784
146.13067242696948
145.9594362543022
149.64638166406542
144.39925990266931
146.18194396622678
144.39610613062658
145.79738787975037
145.17333064794204
145.5046847291972
144.5973278707483
144.8456847957446
145.8687658998648
142.52963368727987
147.08562174609705
144.70685108525535
142.8642744077692
144.76024084290503
144.8445750588138
147.15727739333386
145.70610299339353
148.91175136330065
145.8624339014283
147.03154797651072
147.19415584410186
148.45852849581135
140.43069977861407
144.84769708054947
145.40039058908476
142.37781782134684
144.86676282541538
143.0130505536447
149.12560867107794
146.43174634433535
144.39751003892374
144.9713242512894
144.04740957717064
142.01108695802634
148.78897488009034
147.6449789155267
150.01746195915663
144.74670587140767
144.98962524803
149.38378274740654
141.4436696266514
145.90863548166374
146.9612897483107
148.88482336255544
143.85461475777186
144.63176316126

# Test function

# Function to test the model

In [17]:
# from scipy.io import savemat

def test(H_raw, Di, H_in, e, xTx, xTy, N_test, log):
    # Load the model that we saved at the end of the training loop
    model = model_driven()
    model.load_state_dict(torch.load(best_model, map_location=torch.device('cpu')))

    with torch.no_grad():
        H_o, e_o = model(Di, H_in, e, xTx, xTy)

        nmse = 0
        for j in range (N_test):
            nmse += NMSE(H_o[j], H_raw[j])

        nmse = nmse / N_test
        with Record(log):
            print(format(nmse.item(), '.7f'))

In [18]:
## Generate dataset for test

In [19]:
def LS(DataSet_x, DataSet_y):
    start = timeit.default_timer()
    for i in range (len(DataSet_x)):
        H_hat = np.matmul(
                    np.matmul(
                        np.linalg.pinv(np.matmul(DataSet_x[i].transpose(), DataSet_x[i])),
                        DataSet_x[i].transpose()),
                        DataSet_y[i])
    print(timeit.default_timer() - start)

In [20]:
SNR_min_dB  = 0
SNR_max_dB  = 20
step_dB     = 5
num_dB      = int((SNR_max_dB - SNR_min_dB) / step_dB) + 1

SNR         = np.linspace(SNR_min_dB, SNR_max_dB, num=num_dB)
log         = './model/log_test.txt'

N_test = int(num_samp * 30 / 100)

for i in range (100):
    for snr in SNR:
        # with Record(log):
        #     print(snr)
        DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering = Gen_dataset('test', 'UCyA', 0, 0, N_test)
        H_true, H_raw, H_in, e, xTx, xTy, Di, steering = Input_ISDNN('test', DataSet_x, DataSet_y, DataSet_H, DataSet_HH, DataSet_Steering, N_test)

        # LS(DataSet_x, DataSet_y)
        test(H_raw, Di, H_in, e, xTx, xTy, N_test, log)

<ipython-input-6-3e5f3c45b18a>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dP[i] = abs(IFsig[i])**2
<ipython-input-17-466beddd61d6>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you s

0.0005747
0.0005759
0.0005740
0.0005742


KeyboardInterrupt: 

In [21]:
google.colab import files

files.download("dataset_SISDNN.pkl")

SyntaxError: invalid syntax (<ipython-input-21-14f4914d766c>, line 1)